# Using PhoriaJS for 3D View

#### Example: quick adaptation of external JS libraries. 
( Here we use Phoria.js to generate interactive 3d view in the notebook. )
* add meta [ **app/static/meta/phoria.json** ]:
<code>
{
    "type":"phoria",
    "js": ["phoria/all"],
    "css": ["phoria"],
    "view": "phoria",
    "layout": "phoria",
    "description":"DEFAULT: 3d view with Phoria http://www.kevs3d.co.uk/dev/phoria/",
    "x":"REQUIRE: label to extract variable from data-source",
    "y":"REQUIRE: label to extract variable from data-source",
    "z":"REQUIRE: label to extract variable from data-source",
    "seg":"DEFAULT: None segmentation for colormap",
    "xlabel":"DEFAULT: `x` label",
    "ylabel":"DEFAULT: `y` label",
    "zlabel":"DEFAULT: `z` label",
    "title":"DEFAULT: None",
    "mode":"DEFAULT: scatter [`scatter`,`line`]",
    "stroke":"DEFAULT: 2",
    "rotate":"DEFAULT: true",
    "colormap":"DEFAULT: None or dictionary [ by seg value if `seg` is set ]",
    "grid":"DEFAULT: 10 [ integer or `false` to hide ]",
    "axes":"DEFAULT: true [ show axes ]",
    "width":"DEFAULT: 100%",
    "height":"DEFAULT: 500"
}
</code>
* add script [ **app/src/js/phoria/vis.js** ] to handle local adaptation
* add css [ **app/src/css/phoria.css** ]
* add templates [ **app/views/layout/phoria.hbs**, **app/views/phoria.hbs** ] to refer external resources
* optional shortcat function in **viz/local.py**
<pre>
    def phoria(self, data, **kwargs):
        '''
        quick adaptation of Phoria JS library for rotating 3d view
        '''
        kwargs['type'] = 'phoria'
        if 'seg'  not in kwargs: kwargs['seg'] = False
        if 'axes' not in kwargs: kwargs['axes'] = True
        if 'grid' not in kwargs: kwargs['grid'] = 10
        return self.static(data, kwargs)
</pre>

Ready to go:

In [1]:
import numpy
import pandas
import datetime
import time
import json

from viz import local
viz = local.VizNotebook()

In [2]:
viz.args('phoria')

x               REQUIRE label to extract variable from data-source
y               REQUIRE label to extract variable from data-source
z               REQUIRE label to extract variable from data-source
axes            DEFAULT true [ show axes ]
colormap        DEFAULT None or dictionary [ by seg value if `seg` is set ]
description     DEFAULT 3d view with Phoria http://www.kevs3d.co.uk/dev/phoria/
grid            DEFAULT 10 [ integer or `false` to hide ]
height          DEFAULT 500
mode            DEFAULT scatter [`scatter`,`line`]
rotate          DEFAULT true
seg             DEFAULT None segmentation for colormap
stroke          DEFAULT 2
title           DEFAULT None
width           DEFAULT 100%
xlabel          DEFAULT `x` label
ylabel          DEFAULT `y` label
zlabel          DEFAULT `z` label


In [3]:
# usage motivation
from sklearn.datasets import load_iris
iris = load_iris()
df = pandas.DataFrame(data = numpy.c_[iris['data'], iris['target']], columns = iris['feature_names'] + ['target'])
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [4]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 3)
pca.fit(df[df.columns[:-1]])
C = pandas.DataFrame(pca.transform(df[df.columns[:-1]]), columns=['C1','C2','C3'])
C['target'] = df['target'].apply(lambda t: ['setosa','versicolor','virginica'][int(t)-1])

In [5]:
# data format: dictionary per observation
data = C.to_json(orient='records')
print(json.loads(data)[0])

{'C1': -2.6842071251, 'C3': -0.021511837, 'C2': 0.3266073148, 'target': 'virginica'}


In [6]:
viz.phoria(data, height=800, width='100%',
           x='C1',
           y='C2',
           z='C3',
           seg = 'target',
           colormap = { 'setosa':'#48D1CC', 'versicolor':'#FF69D9', 'virginica':'#FF8C00' },
           stroke = 8,
           grid = 20,
           title='Rotating 3D View')

In [7]:
df = pandas.DataFrame(numpy.zeros((200, 3)), columns=list('ABC'))
df['A'] += df.index.map(lambda i: 10 * i * numpy.cos(0.1 * i))
df['B'] += df.index.map(lambda i: 100 * i * numpy.sin(0.1 * i))
df['C'] += df.index.map(lambda i: 1000 * numpy.cos(0.01 * i))

viz.phoria(df.to_json(orient='records'),
           x='A',
           y='B',
           z='C',
           title='Interactive 3D View',
           rotate = False,
           mode = 'line')

In [ ]:
viz.publish(path_publish='projects/viz/app/static/examples/phoria', path_notebook='projects/viz/examples/PhoriaJS')